In [2]:
import albumentations as A

In [3]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 122kB 7.4MB/s 
     |████████████████████████████████| 952kB 12.8MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65163 sha256=cb0469dcf363a1cadcefa043d6315d96df081e68001bacfd72f67b8ef397962b
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [1]:
from albumentations.pytorch import ToTensorV2

In [2]:
import torch
import albumentations
from albumentations.pytorch import ToTensorV2
import numpy
import torch
import albumentations as A

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/cv_project

/content/drive/MyDrive/cv_project


In [ ]:
!unzip CV_project.zip

Streaming output truncated to the last 5000 lines.
  inflating: CV_project/data/train_masks/0cdf5b5d0ce1_13_mask.gif  
 extracting: CV_project/data/train_masks/0cdf5b5d0ce1_14_mask.gif  
 extracting: CV_project/data/train_masks/0cdf5b5d0ce1_15_mask.gif  
 extracting: CV_project/data/train_masks/0cdf5b5d0ce1_16_mask.gif  
 extracting: CV_project/data/train_masks/0ce66b539f52_01_mask.gif  
 extracting: CV_project/data/train_masks/0ce66b539f52_02_mask.gif  
 extracting: CV_project/data/train_masks/0ce66b539f52_03_mask.gif  
  inflating: CV_project/data/train_masks/0ce66b539f52_04_mask.gif  
 extracting: CV_project/data/train_masks/0ce66b539f52_05_mask.gif  
  inflating: CV_project/data/train_masks/0ce66b539f52_06_mask.gif  
 extracting: CV_project/data/train_masks/0ce66b539f52_07_mask.gif  
 extracting: CV_project/data/train_masks/0ce66b539f52_08_mask.gif  
  inflating: CV_project/data/train_masks/0ce66b539f52_09_mask.gif  
 extracting: CV_project/data/train_masks/0ce66b539f52_10_mask.gif

In [5]:
%cd /content/drive/MyDrive/cv_project/CV_project/

/content/drive/MyDrive/cv_project/CV_project


In [6]:
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from model import UNET
from utils import (
    load_checkpoint,
    save_checkpoint,
    get_loaders,
    check_accuracy,
    save_predictions_as_imgs,
)

# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 3
NUM_WORKERS = 2
IMAGE_HEIGHT = 160  # 1280 originally
IMAGE_WIDTH = 240  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = r"/content/drive/MyDrive/cv_project/CV_project/data/train/train"
TRAIN_MASK_DIR = r"/content/drive/MyDrive/cv_project/CV_project/data/train_masks"
VAL_IMG_DIR = r"/content/drive/MyDrive/cv_project/CV_project/data/val_data"
VAL_MASK_DIR = r"/content/drive/MyDrive/cv_project/CV_project/data/val_masks"

def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())


def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)


    check_accuracy(val_loader, model, device=DEVICE)
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        # check accuracy
        check_accuracy(val_loader, model, device=DEVICE)

        # print some examples to a folder
        save_predictions_as_imgs(
            val_loader, model, folder="/content/drive/MyDrive/cv_project/CV_project/saved/", device=DEVICE
        )


if __name__ == "__main__":
    main()

  0%|          | 0/318 [00:00<?, ?it/s]

Got 198933/883200 with acc 22.52
Dice score: 0.38012680411338806


100%|██████████| 318/318 [31:45<00:00,  5.99s/it, loss=0.111]


=> Saving checkpoint
Got 872927/883200 with acc 98.84
Dice score: 0.973705530166626


100%|██████████| 318/318 [03:54<00:00,  1.36it/s, loss=0.0656]


=> Saving checkpoint
Got 877545/883200 with acc 99.36
Dice score: 0.9857357144355774


100%|██████████| 318/318 [03:55<00:00,  1.35it/s, loss=0.0416]


=> Saving checkpoint
Got 877994/883200 with acc 99.41
Dice score: 0.9868642687797546
